In [353]:
from abc import ABC, abstractmethod

import random
import numpy as np
import pandas as pd

from dataclasses import dataclass

import torch
from torch import nn
from torch.distributions import Categorical
import torch.nn.functional as F

from tqdm.notebook import tqdm
import easyrl.models.diag_gaussian_policy as DiagGaussian

In [354]:

# Stolen from easyrl Categorical policy policy with some modifications
class CategoricalPolicy(nn.Module):
    def __init__(self,
                 body_net,
                 action_dim,
                 in_features=None,
                 ):
        super().__init__()
        self.body = body_net
        if in_features is None:
            for i in reversed(range(len(self.body.fcs))):
                layer = self.body.fcs[i]
                if hasattr(layer, 'out_features'):
                    in_features = layer.out_features
                    break

        self.head = nn.Sequential(nn.Linear(in_features, action_dim), nn.Softmax())

    def forward(self, x=None, body_x=None, **kwargs):
        if x is None and body_x is None:
            raise ValueError('One of [x, body_x] should be provided!')
        if body_x is None:
            body_x = self.body(x, **kwargs)
        if isinstance(body_x, tuple):
            pi = self.head(body_x[0])
        else:
            pi = self.head(body_x)
        action_dist = Categorical(probs=pi)
        return action_dist, body_x

In [355]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

# set random seed
seed = 0
set_random_seed(seed=seed)



In [356]:
class NNetwork(nn.Module):
    def __init__(self, input_dim, action_dim, final_activation=None):
        self.final_activation = final_activation
        super().__init__()
        #### A simple network that takes
        #### as input the history, and outputs the 
        #### distribution parameters.
        self.fcs = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU())
        self.out_layer = nn.Sequential(nn.Linear(64, action_dim))

    def forward(self, ob):
        mid_logits = self.fcs(ob)
        logits = self.out_layer(mid_logits)
        if self.final_activation is not None:
            logits = self.final_activation(logits, dim=-1)
        return logits

# class MishNet(NNetwork):
#     def __init__(*args, mission_shapes, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.final_layers = [nn.Sequential(nn.Linear(64, s)) for s in mission_shapes]
        
#     def forward(self, ob):
#         mid_logits = self.fcs(ob)
#         logits = [fl(mid_logits) for fl in self.final_layers]
#         if self.final_activation is not None:
#             logits = [self.final_activation(logit) for logit in logits]
#         return logits
        
    
    

In [357]:
RED_TEAM_ID = 1
BLUE_TEAM_ID = 0
NUM_PLAYERS = 5
RED_PLAYERS = 2
BLUE_PLAYERS = 3
HIST_SHAPE = 2 * 25 * (3 * NUM_PLAYERS + 5)
SELF_SHAPE = torch.tensor([5])
COMM_SHAPE = 32  # Change freely
WHO_SHAPE = torch.tensor([NUM_PLAYERS])
VOTE_SHAPE =torch.tensor([NUM_PLAYERS])

assert RED_PLAYERS + BLUE_PLAYERS == NUM_PLAYERS

def get_who():
    return NNetwork(SELF_SHAPE + HIST_SHAPE + NUM_PLAYERS*COMM_SHAPE, WHO_SHAPE, nn.functional.softmax)

def get_comm():
    return DiagGaussian.DiagGaussianPolicy(NNetwork(SELF_SHAPE + HIST_SHAPE + WHO_SHAPE, torch.tensor([64])), COMM_SHAPE, in_features=torch.tensor([64]))

def get_miss(mission_shapes = (10,10)):
    model = NNetwork(SELF_SHAPE + WHO_SHAPE + HIST_SHAPE, 64)
    return [CategoricalPolicy(model, mish, 64) for mish in mission_shapes]

def get_vote():
    return CategoricalPolicy(NNetwork(HIST_SHAPE + SELF_SHAPE + WHO_SHAPE, 128), 2, 128)

def get_succ():
    return CategoricalPolicy(NNetwork(HIST_SHAPE + SELF_SHAPE + WHO_SHAPE, 128), 2, 128)



In [358]:
@dataclass
class Agent(ABC):
    
    @abstractmethod
    def comm():
        pass

    @abstractmethod
    def who():
        pass

    @abstractmethod
    def miss():
        pass

    @abstractmethod
    def vote():
        pass

    @abstractmethod
    def succ():
        pass

class RedAgent():

    def __init__(self, lr):
        self.lr = lr

        self.COMM = get_comm()
#         self.who = (lambda *args : every)
        mission_models = get_miss()
        self.MISS = (lambda args : [mission_models[i](args) for i in range(2)])
        self.VOTE = get_vote()
        self.SUCC = get_succ()
        
        # Optimizers
        self.COMM_opt = torch.optim.Adam(self.COMM.parameters(), lr=lr)
        self.MISS_opt2 = torch.optim.Adam(mission_models[0].parameters(), lr=lr)
        self.MISS_opt3 = torch.optim.Adam(mission_models[1].parameters(), lr=lr)
        self.VOTE_opt = torch.optim.Adam(self.VOTE.parameters(), lr=lr)
        self.SUCC_opt = torch.optim.Adam(self.SUCC.parameters(), lr=lr)

class BlueAgent():
    
    def __init__(self, lr):
        self.lr = lr

        self.COMM = get_comm()
        self.WHO = get_who()
        mission_models = get_miss()
        self.MISS = (lambda args : [mission_models[i](args) for i in range(2)])
        self.VOTE = get_vote()
#         self.succ = (lambda *args : torch.distributions.bernoulli.Bernoulli(1))
        
        # Optimizers
        self.COMM_opt = torch.optim.Adam(self.COMM.parameters(), lr=lr)
        self.WHO_opt = torch.optim.Adam(self.WHO.parameters(), lr=lr)
        self.MISS_opt2 = torch.optim.Adam(mission_models[0].parameters(), lr=lr)
        self.MISS_opt3 = torch.optim.Adam(mission_models[1].parameters(), lr=lr)
        self.VOTE_opt = torch.optim.Adam(self.VOTE.parameters(), lr=lr)
    
    

In [359]:
class AvalonEnv():
    def __init__(self):
        super().__init__()
        self.nm = [2, 3, 2, 3, 3, 0]
        self.reset()
    
    def reset(self):
        self.mi = 0
        self.ri = 0
        self.si = 0
        self.li = 0
        self.hist = torch.zeros((2, 25, 20))
        every = [0, 0, 0, 1, 1]
        random.shuffle(every)
        self.every = torch.Tensor(every.copy())        
        self.who = torch.normal(0.5, 0.1, (NUM_PLAYERS, NUM_PLAYERS))
        self.done = False
        self.winning_team = None
        
        # the initial observation of "hist" is all zeros
        # EXCEPT a one at the leader id (self.li) location of the zero-th step
        self.hist[0, 0, self.li] = 1
        self.hist[1, 0, :5] = 1
        
        return self.get_observation()
    
    def get_observation(self):         
        return self.hist, self.every, self.who, self.li, self.si, self.mi, self.nm[self.mi], self.done
    
    def update_who(self, who_m):
        self.who = who_m
        
    def update_miss(self, miss):
        # save to self.hist
        self.hist[0, self.si, 5:10] = miss.detach()
        self.hist[1, self.si, 5:10] = 1
        return self.hist
    
    def update_vote(self, vote):
        # save to self.hist
        self.hist[0, self.si, 10:15] = vote
        self.hist[1, self.si, 10:15] = 1
        
        # check if there are more yeses than noes
        if (vote >= 0.5).sum() > 2:
            # set relevance of only the no mission flag
            self.hist[1, self.si, 15] = 1
        else:
            # set the no mission flag
            self.hist[0, self.si, 15] = 1

            # set the current round
            self.hist[0, self.si, 19] = self.ri
            
            # set the number of failures
            self.hist[0, self.si, 18] = self.hist[0, self.si-1, 18] if self.si else 0
            
            # set relevance
            self.hist[1, self.si, 15:] = 1
            
            if self.ri == 4:
                # game is over, red team wins
                self.winning_team = RED_TEAM_ID
                self.done = True
                
            # update leader
            self.li = (self.li + 1) % 5
            self.hist[0, self.si, self.li] = 1
            self.hist[1, self.si, :5] = 1
            
            # update round id
            self.ri = (self.ri+1) % 5
            
            # update step id
            self.si += 1
        return self.hist
        
    def update_succ(self, succ):
        # set the current round
        self.hist[0, self.si, 19] = self.ri
        
        # set relevance
        self.hist[1, self.si, 16:] = 1
        
        if (succ < 0.5).sum():
            # set the mission failure flag
            self.hist[0, self.si, 17] = 1
            
            # set the number of failures
            self.hist[0, self.si, 18] = self.hist[0, self.si-1, 18] + 1 if self.si else 1
        else:
            # set the mission success flag
            self.hist[0, self.si, 16] = 1
            
            # set the number of failures
            self.hist[0, self.si, 18] = self.hist[0, self.si-1, 18] if self.si else 0
        
        # check if game is over
        if self.hist[0, self.si, 18]  == 3:
            # game is over, red team wins
            self.winning_team = RED_TEAM_ID
            self.done = True
        elif self.mi == 2 + self.hist[0, self.si, 18]:
            # game is over, blue team wins
            self.winning_team = BLUE_TEAM_ID
            self.done = True
            
        # update mission id
        self.mi += 1

        # update round id
        self.ri = 0

        # update step id
        self.si += 1

        # update leader
        self.li = (self.li + 1) % 5
        self.hist[0, self.si, self.li] = 1
        self.hist[1, self.si, :5] = 1

In [360]:
@torch.no_grad()
def miss_players(miss):
    # 0: 0,1
    # 1: 0,2
    # 2: 0,3
    # 3: 0,4
    # 4: 1,2
    # 5: 1,3
    # 6: 1,4
    # 7: 2,3
    # 8: 2,4
    # 9: 3,4
    miss_cat = torch.zeros(NUM_PLAYERS)
    if miss<4:
        miss_cat[0] = 1
        miss_cat[miss+1] = 1
    elif miss<7:
        miss_cat[1] = 1
        miss_cat[miss-2] = 1
    elif miss<9:
        miss_cat[2] = 1
        miss_cat[miss-4] = 1
    else:
        miss_cat[3] = 1
        miss_cat[miss-5] = 1
    return miss_cat

In [400]:
args = getDefaultParams()
@dataclass
class AvalonEngine:
    env: AvalonEnv
    train_episodes: int
    max_epoch: int
    blue: BlueAgent
    red: RedAgent
    trainable_models: list
    gamma: float
    gae_lambda: float
    
    def run(self):
        # Useful for constructing the self vector (self_v)
        self_m = torch.eye(5)
        
        for epoch in tqdm(range(self.max_epoch), desc='epoch'):
            for train_model in tqdm(self.trainable_models, desc='train_model'):
                # print(f'train model = {train_model}')

                # trajectory buffer
                obs_replay_epoch = []
                actions_replay_epoch = []
                log_probs_epoch = []
                stepid_replay_epoch = []
                winning_team = []
                every_replay = []

                for episode in range(self.train_episodes):
                    obs_replay = []
                    actions_replay = []
                    log_probs = []
                    stepid_replay = []

                    hist, every, who, li, si, mi, nm, done = self.env.reset()
                    every_replay.append(every)

                    while not done:
                        # print(f'li, si, mi, ri = {li}, {si}, {mi}, {self.env.ri}')
                        # Flow: communication -> predict who -> decide miss -> voting -> succ/fail -> next round
                        #                                                             -> next round

#                         print(f'Mission {mi} Round {self.env.ri} Step {si}')
                        
                        ''' Communication '''
                        # Initialize communication matrix
                        comm_m = []
                        # Loop over every agent
                        for i in range(5):
                            # create input vector for network                            
                            comm_in = torch.cat((self_m[i], every if every[i] else who[i], hist.flatten()))
                            
#                             if agents == []:
#                                 comm_v = torch.rand(COMM_SHAPE)
#                             else:
                            
                            # Call the COMM network of the current agent and return the communication vector (comm_v)
                            comm_dist, _ = self.blue.COMM(comm_in) if every[i]==BLUE_TEAM_ID else self.red.COMM(comm_in)
                            comm_v = comm_dist.sample()
                
                            # Append it to the communication matrix (comm)
                            comm_m.append(comm_v)
                            # If we are currently training on the COMM network, save it to experience buffer
                            if (train_model == 'comm_red' and every[i] == RED_TEAM_ID) or \
                                    (train_model == 'comm_blue' and every[i] == BLUE_TEAM_ID):
                                obs_replay.append(comm_in)
                                actions_replay.append(comm_v)
                                log_probs.append(comm_dist.log_prob(comm_v))
                                stepid_replay.append(si)
                        # Make the torch.Tensor communication matrix
                        comm_m = torch.cat(comm_m)
                        
#                         print(f'  All the players communicated')

                        ''' Predicting Who '''
                        # Loop over every agent
                        for i in range(NUM_PLAYERS):
                            # continue if the agent is on the red team
                            if every[i] == RED_TEAM_ID:
                                continue
                            # create input vector for network
                            who_in = torch.cat((self_m[i], comm_m, hist.flatten()))

                            # Call the WHO network of the current agent (on the blue team)
                            # and return the who vector (who)
                            who_v = self.blue.WHO(who_in)
                            
                            # Update "who_v" into "who"
                            who[i] = who_v
                            # If we are currently training on the WHO network, save it to experience buffer
                            if train_model == 'who_blue':
                                obs_replay.append(who_in)
                                actions_replay.append(who_v)
                                stepid_replay.append(si)
                        # update the who matrix to the environment
                        self.env.update_who(who)
                        
#                         print(f'  All blue players predicted who')
#                         for i in range(NUM_PLAYERS):
#                             if every[i] == RED_TEAM_ID:
#                                 continue
#                             print(f'    {i} -> {who[i]}')

                        ''' Deciding candidates to go on mission '''
                        # create input vector for "miss" network
                        miss_in = torch.cat((self_m[li], every if every[li] else who[li], hist.flatten()))

                        # Only call the leader
                        miss_dist = self.blue.MISS(miss_in) if every[li]==BLUE_TEAM_ID else self.red.MISS(miss_in)
                        miss = miss_dist[nm - 2][0].sample()
            
                        # If we are currently training on the MISS network, save it tp experience buffer
                        if ('miss_red' in train_model and every[i] == RED_TEAM_ID) or ('miss_blue' in train_model and every[i] == BLUE_TEAM_ID):
                            if str(nm) in train_model:
                                obs_replay.append(miss_in)
                                actions_replay.append(miss)
                                log_probs.append(miss_dist)
                                stepid_replay.append(si)
                        # Update the "miss" vector to the environment
                        miss = miss_players(miss) if nm==2 else 1-miss_players(miss)
                        hist = self.env.update_miss(miss)
                        
#                         print(f'  Leader {li} decides {miss} go on mission')
                            
                        ''' Voting YES/NO for the mission candidates '''
                        # Initialize vote vector
                        vote = []
                        # Loop over every agent   
                        for i in range(5):
                            # create input vector for network
                            vote_in = torch.cat((self_m[i], every if every[i] else who[i], hist.flatten()))

                            # Call the VOTE network of the current agent and return vote_pi
                            vote_dist, _ = self.blue.VOTE(vote_in) if every[i]==BLUE_TEAM_ID else self.red.VOTE(vote_in)
                            vote_pi = vote_dist.sample()
            
                            # Append the voting results to "vote"
                            vote.append(vote_pi)
                            # If we are currently training on the VOTE network, save it to experience buffer
                            if (train_model == 'vote_red' and every[i] == RED_TEAM_ID) or \
                                    (train_model == 'vote_blue' and every[i] == BLUE_TEAM_ID):
                                obs_replay.append(vote_in)
                                actions_replay.append(vote_pi)
                                log_probs.append(vote_dist)
                                stepid_replay.append(si)
                        # Make the torch.Tensor vote vector
                        vote = torch.Tensor(vote)
                        
                        # Update the "vote" vector to the environment
                        hist = self.env.update_vote(vote)
                        
#                         print(f'  Voting results {vote} -> {(vote >= 0.5).sum() > 2}')
                        
    
                        ''' Success/Failure for the mission '''
                        # check if there are more yeses than noes
                        if (vote >= 0.5).sum() > 2:
                            # Initialize succ vector
                            succ = []
                            # Loop over every agent   
                            for i in range(5):
                                if not miss[i]:
                                    continue
                                # create input vector for network
                                succ_in = torch.cat((self_m[i], every if every[i] else who[i], hist.flatten()))

                                # Call the SUCCESS network of the current agent and return succ_i
                                if every[i] == BLUE_TEAM_ID:
                                    succ_i = torch.Tensor([1.0])
                                else:
                                    succ_dist, _ = self.red.SUCC(succ_in)
                                    succ_i = succ_dist.sample()
            
                                # Append the voting results to "vote"
                                succ.append(succ_i)
                                
                                # If we are currently training on the SUCCESS network, save it to experience buffer
                                if train_model == 'succ_red' and every[i] == RED_TEAM_ID:
                                    obs_replay.append(succ_in)
                                    actions_replay.append(succ_i)
                                    log_probs.append(succ_dist)
                                    stepid_replay.append(si)
                            
                        # Make the torch.Tensor succ vector
                        succ = torch.Tensor(succ)
                        
                        # Update the "succ" vector to the environment
                        self.env.update_succ(succ)
                            
#                         print(f'  Mission: {hist[0,self.env.si-1,15:18]} - {hist[0,self.env.si-1,18]} Fails {hist[0,self.env.si-1,19]} Round')
                        
                        hist, every, who, li, si, mi, nm, done = self.env.get_observation()
                        
                    # append the result of each episode into the epoch buffer
                    obs_replay_epoch.append(obs_replay)
                    actions_replay_epoch.append(actions_replay)
                    log_probs_epoch.append(log_probs)
                    stepid_replay_epoch.append(stepid_replay)
                    winning_team.append(self.env.winning_team)
                    last_step = si
                
                obs_all = []
                actions_all = []
                log_probs_all = []
                stepid_all = []
                discounted_rewards = []
                for epi in range(self.train_episodes):
                    if len(stepid_replay_epoch[epi]) > 0:
                        obs_all += obs_replay_epoch[epi]
                        actions_all += actions_replay_epoch[epi]
                        log_probs_all += log_probs_epoch[epi]
                        stepid_all += stepid_replay_epoch[epi]
                        discounted_rewards += compute_discounted_rewards(stepid_replay_epoch[epi], last_step, winning_team[epi], train_model, self.gamma)
                    #####
                
                
                value_function = (lambda x : 0)
                sb = preprocess(obs_all, actions_all, stepid_all, discounted_rewards, value_function)

                model_lookup = {"comm_red": (red.COMM, red.COMM_opt),
                    "miss_red_2" : (red.MISS, red.MISS_opt2),
                    "miss_red_3" : (red.MISS, red.MISS_opt3),
                    "vote_red" : (red.VOTE, red.VOTE_opt), 
                    "succ_red" : (red.SUCC, red.SUCC_opt),
                    "who_blue" : (blue.WHO, blue.WHO_opt),
                    "comm_blue" : (blue.COMM, blue.COMM_opt),
                    "miss_blue_2" : (blue.MISS, blue.MISS_opt2),
                    "miss_blue_3" : (blue.MISS, blue.MISS_opt3),
                    "vote_blue" : (blue.VOTE, blue.VOTE_opt)}

                acmodel, optimizer = model_lookup[train_model]

                if train_model != "who_blue":
                    update_parameters_ppo(optimizer, acmodel, sb, args, train_model)
                else:
                    optimizer.zero_grad()
                    loss = 0
                    for epi in range(self.train_episodes):
                        loss += ((acmodel(obs_all[epi]) - every_replay[epi]) ** 2).sum()
                    loss.backward()
                    optimizer.step()

                # end of self.train_episodes episodes
                # # obs_replay, actions_replay, stepid_replay, winning_team
                # if len(stepid_replay) > 0:
                #     discounted_rewards = compute_discounted_rewards(stepid_replay, winning_team, train_model, self.gamma)
                #     advantage_gae = compute_advantage_gae([0.0] * len(stepid_replay), discounted_rewards, stepid_replay, self.gae_lambda, self.gamma, train_model)
                # sb['obs', 'action', 'advantage_gae' or 'advantage', 'discounted_reward']
                    

In [401]:
# #@title
# def compute_advantage_gae(values, discounted_rewards, stepid_replay, gae_lambda, gamma, train_model):
#     rewards = [x if abs(x) == 1 else 0 for x in discounted_rewards]
#     # print(rewards)
#     # print(discounted_rewards)

#     reps = 0
#     if stepid_replay[0] != 0:
#         print(train_model)
#         print(stepid_replay)
#     while stepid_replay[reps] == 0:
#         reps += 1

#     rewards_single = torch.Tensor(rewards[::reps])
#     values_single = torch.Tensor(values[::reps])

#     end_of_epi = np.arange(len(rewards_single))[rewards_single != 0].astype(int)
    
#     # print(end_of_epi)
#     # print(values_single)

#     last_end = -1
#     total_advs = []
#     for end in end_of_epi:
#         # calculate advantage_gae from index last_end+1 ~ end\
#         theta = rewards_single[last_end+1:end+1] - values_single[last_end+1:end+1]
#         theta[:-1] += gamma * values_single[last_end+2:end+1]
#         gl = gamma * gae_lambda
#         advantages = torch.tensor([
#             (theta[i:] * (gl ** torch.arange(end - last_end -i))).sum() for i in range(end - last_end)
#         ])

#         # print(f'last_end: {last_end} ~ end: {end} => {len(advantages)}')
#         # print(advantages)
#         total_advs.append(advantages)
#         last_end = end
#     ############################################################################
    
#     total_advs = torch.concat(total_advs)
#     torch.repeat_interleave(total_advs, 2)

#     # print(total_advs)
#     # print(torch.repeat_interleave(total_advs, 2))
#     # assert 0

#     return total_advs

In [402]:
# args = getDefaultParams()
def compute_discounted_rewards(stepid_replay, last_step, winning_team, train_model, gamma):

    # last_step = 10
    # stepid_replay = [0, 0, 1, 3, 5, 5, 6, 6, 6, 8]

    # Calculate the discounted rewards
    win_team_color = 'red' if winning_team == RED_TEAM_ID else 'blue'
    R = 1 if win_team_color in train_model else -1
    discounted_rewards = [R * (gamma ** (last_step - 1 - stepid_replay[-1]))]
    curr_step = stepid_replay[-1]

    for si in range(len(stepid_replay) - 2, -1, -1):
        discounted_rewards += [discounted_rewards[-1] * (gamma ** (curr_step - stepid_replay[si]))]
        curr_step = stepid_replay[si]

    # stepN = len(stepid_replay)
    # win_team_color = 'red' if winning_team[-1] == RED_TEAM_ID else 'blue'
    # discounted_rewards = [1 if win_team_color in train_model else -1]

    # for i in range(stepN - 2, -1, -1):
    #     if stepid_replay[i] == stepid_replay[i+1]:
    #         # the current step of the i-th index is the same as the step of the i+1-th index
    #         discounted_rewards.append(discounted_rewards[-1])
    #     elif stepid_replay[i] != 0 and stepid_replay[i+1] == 0:
    #         # the current step of the i-th index is the last step of a new episode
    #         win_team_color = 'red' if winning_team[i] == RED_TEAM_ID else 'blue'
    #         discounted_rewards.append(1 if win_team_color in train_model else -1)
    #     else:
    #         # the current step of the i-th index is the step of the i+1-th index minus 1
    #         discounted_rewards.append(discounted_rewards[-1] * gamma)

    discounted_rewards.reverse()
    return discounted_rewards


def preprocess(obs_replay, actions_replay, stepid_replay, discounted_rewards, value_function):
    # Example:
    # preprocess(*full_replay_buffer, range(10), (lambda x: 0))

    # print(len(obs_replay))
    # print(len(stepid_replay))

    assert len(obs_replay) == len(actions_replay)
    assert len(obs_replay) == len(stepid_replay)
    assert len(discounted_rewards) == len(obs_replay)
    discounted_rewards = torch.Tensor(discounted_rewards)
    # print(obs_replay)
    obs_replay = torch.stack(obs_replay)
    actions_replay = torch.stack(actions_replay)
    # obs_replay = torch.Tensor(obs_replay)
    sb = {
        "obs" : obs_replay,
        "action" : actions_replay,
        "advantage" : discounted_rewards - value_function(obs_replay),
        "discounted_reward" : discounted_rewards,
    }
    return sb



def update_parameters_ppo(optimizer, acmodel, sb, args, train_model):
    def _compute_policy_loss_ppo(logp, old_logp, entropy, advantages):
        policy_loss, approx_kl = 0, 0

        ### TODO: implement PPO policy loss computation (30 pts).  #######
        logr = logp - old_logp
        ratios = torch.exp(logr)
        
        surr1 = ratios * advantages
        surr2 = torch.clamp(ratios, 1-args['clip_ratio'], 1+args['clip_ratio']) * advantages
        
        policy_loss = (-torch.min(surr1, surr2) -args['entropy_coef']*entropy).mean()
        
        # approx_kl = torch.sum(torch.exp(logp) * logr)
        #approx_kl = torch.nn.functional.kl_div(logp, old_logp)

        # approx_kl = ((logr.exp() - 1) - logr).sum()
        approx_kl = ((logr ** 2) / 2).sum()
        
        ##################################################################
        
        return policy_loss, approx_kl
    
    def _compute_value_loss(values, returns):
        ### TODO: implement PPO value loss computation (10 pts) ##########
        value_loss = F.mse_loss(values.squeeze(-1), returns).mean() #(values - returns).pow(2).mean()
        ##################################################################

        return value_loss

    if 'miss' in train_model:
        if "2" in train_model:
            ((dist, _), _) = acmodel(sb['obs'])
        else:
            (_, (dist, _)) = acmodel(sb['obs'])
    else:
        dist, values = acmodel(sb['obs'])
    
    # print(dist)
    
    old_logp = dist.log_prob(sb['action']).detach()
    logp = dist.log_prob(sb['action'])
    dist_entropy = dist.entropy()
    
    advantage = sb['advantage_gae'] if args['use_gae'] else sb['advantage']
    
    policy_loss, _ = _compute_policy_loss_ppo(logp, old_logp, dist_entropy, advantage)

    values = torch.zeros_like(sb['discounted_reward'])

    value_loss = _compute_value_loss(values, sb['discounted_reward'])

    for i in range(args['train_ac_iters']):
        if 'miss' in train_model:
            if "2" in train_model:
                ((dists, _), _) = acmodel(sb['obs'])
            else:
                (_, (dists, _)) = acmodel(sb['obs'])
        else:
            dists, values = acmodel(sb['obs'])
            
        logp = dists.log_prob(sb['action'])
        dist_entropy = dists.entropy()
        
        optimizer.zero_grad()
        
        loss_pi, approx_kl = _compute_policy_loss_ppo(logp, old_logp, dist_entropy, advantage)


        values = torch.zeros_like(sb['discounted_reward'])
        loss_v = _compute_value_loss(values, sb['discounted_reward'])

        loss = loss_v + loss_pi
        if approx_kl > 1.5 * args['target_kl']:
            break
        
        loss.backward(retain_graph=True)
        optimizer.step()
    
    update_policy_loss = policy_loss.item()
    update_value_loss = value_loss.item()

    logs = {
        "policy_loss": update_policy_loss,
        "value_loss": update_value_loss,
    }

    return logs

# PROBLEMS TO FIX
# How to deal with the optimizer for MISS? Do we need to train each model separetely?
# Need supervised learning for WHO

In [403]:
trainable_models=['comm_red', 'comm_blue', 'who_blue', 'miss_red_2', 'miss_red_3', 'miss_blue_2', 'miss_blue_3', 'vote_red', 'vote_blue', 'succ_red']

set_random_seed(0)

lr = 0.00025
gamma = 0.99
gae_lambda = 0.95
blue = BlueAgent(lr=lr)
red = RedAgent(lr=lr)

my_env = AvalonEnv()
engine = AvalonEngine(env=my_env, blue=blue, red=red, train_episodes=100, max_epoch=10,
                      trainable_models=trainable_models, gamma=gamma, gae_lambda=gae_lambda)
engine.run()

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

...

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

train_model:   0%|          | 0/10 [00:00<?, ?it/s]

In [404]:
def getDefaultParams():
    # here goes the default parameters for the agent
    config = dict(
        # env=env, the agent does not need to ahve access to the env because there is an engine
        learning_rate=0.00025,
        gamma=0.99,
        memory_size=200000,
        initial_epsilon=1.0,
        min_epsilon=0.1,
        max_epsilon_decay_steps=150000,
        warmup_steps=500,
        target_update_freq=2000,
        batch_size=32,
        device=None,
        disable_target_net=False,
        enable_double_q=False,
        use_gae=False,
        train_ac_iters=5,
        target_kl=0.01,
        clip_ratio=0.2,
        entropy_coef=0.01
    )
    return config

        # def __init__(self,
        #         score_threshold=0.93,
        #         discount=0.995,
        #         lr=1e-3,
        #         max_grad_norm=0.5,
        #         log_interval=10,
        #         max_episodes=2000,
        #         gae_lambda=0.95,
        #         use_critic=False,
        #         clip_ratio=0.2,
        #         target_kl=0.01,
        #         train_ac_iters=5,
        #         use_discounted_reward=False,
        #         entropy_coef=0.01,
        #         use_gae=False):